# Top Brass Trophy problem (Ex. 5.1 in Rardin'98)

Top Brass Trophy Company makes large championship trophies for youth athletic leagues. At the moment, they are planning production for fall sports: football and soccer. Each football trophy has a wood base, an engraved plaque, a large brass football on top, and returns 12 dollars in profit. Soccer trophies are similar except that a brass soccer ball is on top, and the unit profit is only 9 dollars. Since the football has an asymmetric shape, its base requires 4 board feet of wood; the soccer base requires only 2 board feet. At the moment there are 1000 brass footballs in stock, 1500 soccer balls, 1750 plaques, and 4800 board feet of wood. What trophies should be produced from these supplies to maximize total profit assuming that all that are made can be sold?

### Implementing the model in Julia

In [1]:
using Pkg
Pkg.build("HiGHS")
Pkg.add("HiGHS")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [2]:
# Putting together the model

using JuMP

m = Model()

# (We'll just set up the model here and define the solver later)

@variable(m, 0 <= f <= 1000)           # football trophies
@variable(m, 0 <= s <= 1500)           # soccer trophies
@constraint(m, 4f + 2s <= 4800)        # total board feet of wood
@constraint(m, f + s <= 1750)          # total number of plaques
@objective(m, Max, 12f + 9s)           # maximize profit

12 f + 9 s

In [3]:
# Printing the model
print(m)

In [4]:
# An alternative way of printing the model
println(m)

Max 12 f + 9 s
Subject to
 4 f + 2 s ≤ 4800.0
 f + s ≤ 1750.0
 f ≥ 0.0
 s ≥ 0.0
 f ≤ 1000.0
 s ≤ 1500.0



Now let's solve it, and print the results!

In [5]:
using HiGHS
# @time set_optimizer(m, HiGHS.Optimizer)
set_optimizer(m, HiGHS.Optimizer)
optimize!(m)

println("The total number of football trophies will be ", value(f))
println("The total number of soccer   trophies will be ", value(s))
println("Total profit will be \$", objective_value(m))

Running HiGHS 1.4.0 [date: 1970-01-01, git hash: bcf6c0b22]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
2 rows, 2 cols, 4 nonzeros
2 rows, 2 cols, 4 nonzeros
Presolve : Reductions: rows 2(-0); columns 2(-0); elements 4(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          2     1.7700000000e+04 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  1.7700000000e+04
HiGHS run time      :          0.00
The total number of football trophies will be 650.0
The total number of soccer   trophies will be 1100.0
Total profit will be $17700.0


Another way of implementing the model, separating the data and the model:

In [6]:
# the types of trophies produced
sports = [:football, :soccer]

# wood required for each type of trophy (in board feet)
wood   = Dict( :football => 4, :soccer => 2)

# plaques required for each type of trophy
plaques = Dict( :football => 1, :soccer => 1)

# profit made for each trophy
profit = Dict( :football => 12, :soccer => 9)

# quantities in stock for each ingredient
num_wood     = 4800
num_plaques  = 1750
num_football = 1000
num_soccer   = 1500
;

Here we give names in the model to the plaque constraint, the wood constraint, and the objective.

In [7]:
using JuMP, HiGHS
m1 = Model()

@variable(m1, trophies[sports] >= 0 )    # "trophies" is a dictionary indexed over sports

@expression(m1, tot_plaques, sum(trophies[i] * plaques[i] for i in sports) )
@expression(m1, tot_wood,    sum(trophies[i] * wood[i]    for i in sports) )
@expression(m1, tot_profit,  sum(trophies[i] * profit[i]  for i in sports) )

@constraint(m1, trophies[:soccer] <= num_soccer )      # maximum number of soccer balls
@constraint(m1, trophies[:football] <= num_football )  # maximum number of footballs
@constraint(m1, tot_plaques <= num_plaques )           # maximum number of plaques
@constraint(m1, tot_wood    <= num_wood )              # maximum amount of wood

@objective(m1, Max, tot_profit)

12 trophies[football] + 9 trophies[soccer]

Now we solve this version of the model.

In [8]:
set_optimizer(m1, HiGHS.Optimizer)
optimize!(m1)

println("The total number of football and soccer throphies will be ", [JuMP.value(trophies[i]) for i in sports])
println("Total profit will be \$", JuMP.value(tot_profit))
println("Total wood used is ", JuMP.value(tot_wood), " board feet")
println("Total number of plaques used is ", JuMP.value(tot_plaques))

Running HiGHS 1.4.0 [date: 1970-01-01, git hash: bcf6c0b22]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
2 rows, 2 cols, 4 nonzeros
2 rows, 2 cols, 4 nonzeros
Presolve : Reductions: rows 2(-2); columns 2(-0); elements 4(-2)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          2    -1.7700000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  1.7700000000e+04
HiGHS run time      :          0.00
The total number of football and soccer throphies will be [650.0, 1100.0]
Total profit will be $17700.0
Total wood used is 4800.0 board feet
Total number of plaques used is 1750.0


### Solving the problem with various Linear Programming solvers!

Solve the model and print the solution:

In [9]:
# Let's first add some solvers

using Pkg
Pkg.add("ECOS")
Pkg.add("SCS")


using JuMP, ECOS, SCS

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


Let's compare some solvers! (Note that they run faster after the first time!)

In [12]:
# ECOS

m = Model()

@variable(m, 0 <= f <= 1000)           # football trophies
@variable(m, 0 <= s <= 1500)           # soccer trophies
@constraint(m, 4f + 2s <= 4800)        # total board feet of wood
@constraint(m, f + s <= 1750)          # total number of plaques
@objective(m, Max, 12f +9s)           # maximize profit

set_optimizer(m, ECOS.Optimizer)
@time optimize!(m)
println(termination_status(m))
println("Build ", value(f), " football trophies.")
println("Build ", value(s), " soccer trophies.")
println("Total profit will be \$", objective_value(m))


  0.002220 seconds (973 allocations: 63.281 KiB)
OPTIMAL
Build 649.9999988187323 football trophies.
Build 1100.000000607336 soccer trophies.
Total profit will be $17699.999991290813

ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -1.586e+04  -3.576e+04  +1e+04  3e-06  3e-01  1e+00  1e+03    ---    ---    1  1  - |  -  - 
 1  -1.765e+04  -1.891e+04  +7e+02  2e-07  2e-02  1e+01  1e+02  0.9369  1e-02   0  0  0 |  0  0
 2  -1.769e+04  -1.773e+04  +2e+01  6e-09  7e-04  1e+00  3e+00  0.9820  1e-02   0  0  0 |  0  0
 3  -1.770e+04  -1.770e+04  +2e-01  6e-11  8e-06  1e-02  3e-02  0.9890  1e-04   1  0  0 |  0  0
 4  -1.770e+04  -1.770e+04  +2e-03  7e-13  9e-08  1e-04  4e-04  0.9890  1e-04   1  0  0 |  0  0
 5  -1.770e+04  -1.770e+04  +3e-05  8e-15  1e-09  2e-06  4e-06  0.9890  1e-04   1  0  0 |  0  0

OPTIMAL (within feastol=9.7e-10, reltol=1.5e-09, abstol

In [15]:
# SCS

# this time, defined the optimizer when initializing the model
m = Model(SCS.Optimizer)

@variable(m, 0 <= f <= 1000)           # football trophies
@variable(m, 0 <= s <= 1500)           # soccer trophies
@constraint(m, 4f + 2s <= 4800)        # total board feet of wood
@constraint(m, f + s <= 1750)          # total number of plaques
@objective(m, Max, 12f +9s)           # maximize profit

@time optimize!(m)
println(termination_status(m))
println("Build ", value(f), " football trophies.")
println("Build ", value(s), " soccer trophies.")
println("Total profit will be \$", objective_value(m))


  0.000864 seconds (839 allocations: 53.367 KiB)
OPTIMAL
Build 649.7534130603916 football trophies.
Build 1100.306424402705 soccer trophies.
Total profit will be $17699.798776349046
------------------------------------------------------------------
	       SCS v3.2.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 2, constraints m: 6
cones: 	  l: linear vars: 6
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 8, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 4.80e+03  1.20e+01  3.13e+

In [16]:
# INSTALLING GUROBI AND GETTING A LICENSE: UPDATED 1/20/23

# Go to https://github.com/jump-dev/Gurobi.jl

# Scroll down to "First Obtain a License..." and click through to Gurobi's web site.

# After establishing an account with Gurobi, go to the dropdown menu "Downloads and Licenses" and select "Academic License"

# run "grbgetkey" as instructed, with the license number provided. When it asks which directory to store the license in,
# I type "/Library/gurobi1000"

# Now I run this:

ENV["GUROBI_HOME"] = "/Library/gurobi1000/macos_universal2"
ENV["GRB_LICENSE_FILE"] = "/Library/gurobi1000/gurobi.lic"
import Pkg
Pkg.add("Gurobi")
Pkg.build("Gurobi")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
    Building Gurobi → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/82a44a86f4dc4fa4510c9d49b0a74d3d73914d5c/build.log`


In [19]:
# run the Top Brass model with Gurobi
using Gurobi

# this time, defined the optimizer when initializing the model
m = Model(Gurobi.Optimizer)

@variable(m, 0 <= f <= 1000)           # football trophies
@variable(m, 0 <= s <= 1500)           # soccer trophies
@constraint(m, 4f + 2s <= 4800)        # total board feet of wood
@constraint(m, f + s <= 1750)          # total number of plaques
@objective(m, Max, 12f +9s)           # maximize profit

@time optimize!(m)
println(termination_status(m))
println("Build ", value(f), " football trophies.")
println("Build ", value(s), " soccer trophies.")
println("Total profit will be \$", objective_value(m))

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-15
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x1588307c
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [9e+00, 1e+01]
  Bounds range     [1e+03, 2e+03]
  RHS range        [2e+03, 5e+03]
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1600000e+04   3.875000e+02   0.000000e+00      0s
       2    1.7700000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.770000000e+04

User-callback calls 42, time in user-callback 0.00 sec
  0.000669 seconds (231 allocations: 13.797 KiB)
OPTIMAL
Build 650.0 football trophies.
Build 1100.